In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from environment.env import Environment 
# Disable warnings
import warnings
warnings.filterwarnings("ignore")

# Load the trained model
MODEL = "PPO_31"
save_dir = f"./logs/{MODEL}"
model = PPO.load(f"logs/ppo/model.zip")

# Create and register the environment
gym.register(id="Environment-v0", entry_point=Environment)
env = gym.make("Environment-v0")

def evaluate_for_data_rate(model, env, target_rate, num_episodes=10):
    total_power = 0
    for _ in range(num_episodes):
        obs = env.reset()
        env.min_rate = target_rate  # Set the target rate
        done = False
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            if done or truncated:
                # Sum the individual power values
                episode_power = info['P_1'] + info['P_2'] + info['P_3']
                total_power += episode_power
                break
    return total_power / num_episodes

# Evaluate for different data rates
data_rates = np.linspace(50e6, 500e6, 10)  # 50 Mbps to 500 Mbps
required_powers = []

for rate in data_rates:
    avg_power = evaluate_for_data_rate(model, env, rate)
    required_powers.append(avg_power)
    print(f"Data Rate: {rate/1e6:.5f} Mbps, Required Power: {avg_power:.5f} dB")

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(data_rates / 1e6, required_powers, 'b-o')
plt.xlabel('Data Rate (Mbps)')
plt.ylabel('Required Power (dBm)')
plt.title('Required Power vs Data Rate')
plt.grid(True)
plt.show()